In [4]:
# -*- coding: utf-8 -*-

from __future__ import division

# 引入常规库函数
import numpy as np
import pandas as pd

# 引入统计函数库
# 网上此种引入方法比较普遍
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

# 引入Kalman函数库
from pykalman import KalmanFilter


# 引入纯随机序列检测，只有是非纯随机序列，才能进行检测
from statsmodels.stats.diagnostic import acorr_ljungbox


In [5]:
# 股票名：get_pairs
# 输入参数：
# 1、current_date：提取数据截止的时间点
# 2、ncount：提取的样本数
# 返回值：
# pandas数组，包含交易的股票配对信息、beta、alpha、随机性、平稳性、category等信息
# 功能：
# 利用聚宽的行业分类、价格信息，按照行业分类提取相应的股票对，同时比较节约时间

def get_pairs(current_date,ncount):
    # 存储结果的pandas数组
    ret = pd.DataFrame()

    # 按照聚宽的行业分类，进行相应的数据提取工作
    # 参考聚宽申万二级行业列表
    industry_list = ["HY401","HY402","HY403","HY404","HY405","HY406","HY407","HY408","HY409","HY410", \
                 "HY411","HY412","HY413","HY414","HY415","HY416","HY417","HY418","HY419","HY420", \
                 "HY421","HY422","HY423","HY424","HY425","HY426","HY427","HY428","HY429","HY432", \
                 "HY433","HY434","HY435","HY436","HY437","HY438","HY439","HY440","HY441","HY442", \
                 "HY443","HY444","HY445","HY446","HY447","HY448","HY449","HY450","HY451","HY452", \
                 "HY453","HY454","HY455","HY457","HY458","HY459","HY460","HY461","HY462","HY463", \
                 "HY464","HY465","HY466","HY467","HY468","HY469","HY470","HY471","HY472","HY473", \
                 "HY474","HY476","HY477","HY478","HY479","HY480","HY481","HY483","HY484","HY485", \
                 "HY486","HY487","HY488","HY489","HY491","HY492","HY493","HY494","HY496","HY497", \
                 "HY500","HY501","HY504","HY505","HY506","HY509","HY510","HY511","HY512","HY513", \
                 "HY514","HY515","HY516","HY517","HY518","HY519","HY520","HY521","HY523","HY524", \
                 "HY525","HY526","HY527","HY528","HY529","HY530","HY531","HY570","HY571","HY572", \
                 "HY573","HY574","HY576","HY578","HY579","HY587","HY588","HY591","HY593","HY595", \
                 "HY596","HY597","HY598","HY601"]

    # 证监会行业列表
    industry_list = ["A01","A02","A03","A04","A05","B06","B07","B08","B09","B10","B11", \
                 "B12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22", \
                 "C23","C24","C25","C26","C27","C28","C29","C30","C31","C32","C33", \
                 "C34","C35","C36","C37","C38","C39","C40","C41","C42","C43","D44", \
                 "D45","D46","E47","E48","E49","E50","F51","F52","G53","G54","G55", \
                 "G56","G57","G58","G59","G60","H61","H62","I63","I64","I65","J66", \
                 "J67","J68","J69","K70","L71","L72","M73","M74","M75","N76","N77", \
                 "N78","O79","O80","O81","P82","Q83","Q84","R85","R86","R87","R88","R89","S90"]


    # 开始进行行业循环，提取相应的
    for industry_list_i in industry_list:

        print "开始计算：%s"%str(industry_list_i)

        # 清空存储数据
        # 用于整理数据，中间暂存每个配对对应的pearson、beta等相关信息
        data_pd = pd.DataFrame()

        # 清空存储配对数据，与上面的data_pd相同，为了方便操作
        pairs = []

        # 清空数据
        stock_list = []

        # 获取行业列表
        stock_list = get_industry_stocks(industry_list_i, date= current_date)


        # 获取核心的分析数据
        # cor_data_collect是一个核心函数
        # cor_data的数据结构：
        # 字典结构，键为（pair1，pair2）元组，值为数组[beta,alpha,adf_factor,stochastic]
        cor_data = {}
        cor_data = cor_data_collect(stock_list,day_count = ncount,end_date = current_date)
        
        # 剔除一部分数据不相关的数据
        # 目标为：adf检测为假的数据
        for k,v in cor_data.items():

            # 如果长度小于3，说明没有进行Adf等后续的检测，所以没有必要进行进一步的检查
            # 在此增加判断标准，确定以什么标准进行添加
            # 标准：残差为平稳序列，随机序列
            if (v[2] < 0.05) and (v[3] > 0.05): 
                print "找到匹配数据：%s"%(str(k))
                # 增加了行业industry数据
                pairs.append([k,v,industry_list_i])

        # 对配对数据进行整理，合并所有数据
        data_pd = pd.concat([pd.DataFrame([p[0] for p in pairs],columns = ['p1','p2']), \
                            pd.DataFrame([p[1] for p in pairs],columns = ['beta','alpha','adf_factor','stochastic']),\
                            pd.DataFrame([p[2] for p in pairs],columns = ['category'])], \
                            axis = 1)

        
        # 写入文件
        data_pd.to_csv(industry_list_i)
        
        # 合并数据
        # 如果需要，在合并数据的时候可以进行数据筛选
        ret = pd.concat([ret,data_pd],axis = 0)
    
    return ret


In [6]:

# 函数名：cor_data_collect
# 输入参数：
# 1、stock_list：比对的股票列表
# 2、day_count：选取的每支股票的样本数
# 3、end_date：终止时间
# 输出参数：
# ret：
# 字典结构，键为（pair1，pair2）元组，值为数组[beta,alpha,adf_factor,stochastic]
# 功能：
# 从所给出的stock_list中，两两配对，计算相关系数
def cor_data_collect(stock_list,day_count,end_date):
    # 对应股票列表中的价格指数
    pdata = get_price(stock_list, count = day_count, end_date=end_date, frequency='daily', fields='close',fq = "pre")['close']

    # 进行数据处理
    pdata = pdata.fillna(0)

    # 找到列数
    n = pdata.shape[1]

    # 找到列名
    keys = list(pdata.keys())

    
    # 最终返回的存储结果
    ret = {}
    
    # 输出处理进度
    print "行业内合计股票数为：%d"%int(n)
    
    # 逐项配对，进行匹配验证
    # 按照获取的股票数量
    for i in range(n):
        # 进度提示
        print "进展到第%d"%int(i)
        
        # 对数据进行循环比对
        # 因为kalman方程有方向性，所以i、j都要覆盖
        for j in range(n):
            
            # 进度提示
            # print "正在进行：%s，%s"%(str(keys[i]),str(keys[j]))
            
            # 赋初值，数据为价格
            S1 = pdata[keys[i]]
            S2 = pdata[keys[j]]

            ################    开始进行处理，个性化的处理   ########################
            
            # 1、赋初值,以二者股票的元组为键值
            ret[(keys[i],keys[j])] = []

            try:
                # 2、计算beta、alpha
                # 计算beta、alpha

                # 按照定义：S1为自变量，S2为因变量
                beta_kf = kalman_beta(S1,S2)
                beta = kalman_beta(S1,S2)[0]
                alpha = kalman_beta(S1,S2)[1]

                ret[(keys[i],keys[j])].append(beta)
                ret[(keys[i],keys[j])].append(alpha)

                # 计算残差的差分值
                res = S2 - np.dot(sm.add_constant(S1, prepend=False), beta_kf)
                res_diff = res - res.shift(1)
                res_diff = res_diff.fillna(0)

                # 3、计算平稳性
                ret[(keys[i],keys[j])].append(Cadf_test(res_diff))

                # 4、计算残差的
                ret[(keys[i],keys[j])].append(test_stochastic(res_diff))
            except:
                print "配对出错：%s,%s"%(str(keys[i]),str(keys[j]))
                continue
                
    return ret

In [ ]:
# S1为自变量，S2为因变量
def kalman_beta(S1,S2):
    
    # 观察矩阵
    # 注意：
    # 1、观察到的是sec1数据，sec1是自变量x，sec2是因变量y
    # 2、需要使用add_constant来模拟alpha
    # 3、需要使用np.newaxis来增加维度
    
    obs_mat = sm.add_constant(S1.values, prepend=False)[:, np.newaxis]

    kf = KalmanFilter(n_dim_obs=1, n_dim_state=2, # y is 1-dimensional, (alpha, beta) is 2-dimensional
                  initial_state_mean=np.ones(2),
                  initial_state_covariance=np.ones((2, 2)),
                  transition_matrices=np.eye(2),  # 不发生变化，都是单位矩阵
                  observation_matrices=obs_mat,   # 观察矩阵
                  observation_covariance=10**2,
                  transition_covariance=0.01**2 * np.eye(2))
    
    # 相当于使用sec2来进行训练，模拟出beta、alpha
    state_means, state_covs = kf.filter(S2[:, np.newaxis])
    
    # 第一个参数为beta，第二个参数为alpha
    return state_means[-1]



# 函数名：Cadf_test
# 输入参数：
# 1、res_pd:pandas数组，index为日期，列名为“res”
# 输出参数：
# P value：返回test的p值，用于后续监测
# null hypothesis of the Augmented Dickey-Fuller is that there is a unit root

def Cadf_test(res):

    # 使用adf计算adf的值
    cadf = ts.adfuller(res)
    
    return cadf[1]


# 重要：只有时间序列不是一个白噪声（纯随机序列）的时候，该序列才可做分析
# 函数名：test_stochastic
# 输入参数：
# 1、res_pd:pandas数组，index为日期，列名为“res”
# 输出参数：
# P value：返回test的p值，用于后续监测
# Ljung-Box test for no autocorrelation
# 纯随机性检验,p值小于5%,序列为非白噪声
# H0: 原本的数据都是纯随机序列
# 用于检验某个时间段内的一系列观测值是不是随机的独立观测值
# 如果观测值并非彼此独立，一个观测值可能会在 i 个时间单位后与另一个观测值相关，形成一种称为自相关的关系
# 自相关可以削减基于时间的预测模型（例如时间序列图）的准确性，并导致数据的错误解释。

def test_stochastic(ts):
    p_value = acorr_ljungbox(ts)[1] #lags可自定义
    return p_value[0]

In [ ]:
get_pairs('2018-10-1',500)

开始计算：A01
行业内合计股票数为：17
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
进展到第11
进展到第12
进展到第13
进展到第14
进展到第15
进展到第16
找到匹配数据：('300189.XSHE', '300087.XSHE')
找到匹配数据：('600371.XSHG', '600354.XSHG')
找到匹配数据：('600313.XSHG', '600313.XSHG')
找到匹配数据：('600540.XSHG', '600108.XSHG')
找到匹配数据：('600672.XSHG', '600598.XSHG')
找到匹配数据：('300511.XSHE', '300087.XSHE')
找到匹配数据：('600540.XSHG', '600762.XSHG')
找到匹配数据：('300511.XSHE', '600540.XSHG')
找到匹配数据：('600762.XSHG', '300189.XSHE')
找到匹配数据：('600354.XSHG', '300189.XSHE')
找到匹配数据：('600506.XSHG', '300189.XSHE')
找到匹配数据：('600371.XSHG', '002041.XSHE')
找到匹配数据：('600359.XSHG', '300511.XSHE')
找到匹配数据：('600672.XSHG', '600540.XSHG')
找到匹配数据：('600313.XSHG', '601118.XSHG')
找到匹配数据：('600540.XSHG', '300087.XSHE')
找到匹配数据：('000998.XSHE', '600108.XSHG')
找到匹配数据：('000998.XSHE', '300087.XSHE')
找到匹配数据：('600762.XSHG', '002041.XSHE')
找到匹配数据：('600108.XSHG', '600371.XSHG')
找到匹配数据：('002772.XSHE', '601118.XSHG')
找到匹配数据：('300087.XSHE', '002772.XSHE')
找到匹配数据：('000998.XSHE', '002041.XS

进展到第1
进展到第2
进展到第3
找到匹配数据：('000592.XSHE', '000592.XSHE')
找到匹配数据：('002200.XSHE', '000592.XSHE')
找到匹配数据：('002200.XSHE', '002200.XSHE')
找到匹配数据：('000592.XSHE', '002200.XSHE')
找到匹配数据：('600265.XSHG', '000592.XSHE')
找到匹配数据：('002200.XSHE', '600265.XSHG')
找到匹配数据：('600265.XSHG', '002200.XSHE')
开始计算：A03
行业内合计股票数为：12
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
进展到第11
找到匹配数据：('002714.XSHE', '000735.XSHE')
找到匹配数据：('300498.XSHE', '002477.XSHE')
找到匹配数据：('300106.XSHE', '002234.XSHE')
找到匹配数据：('002746.XSHE', '002299.XSHE')
找到匹配数据：('600975.XSHG', '002234.XSHE')
找到匹配数据：('002321.XSHE', '600965.XSHG')
找到匹配数据：('600975.XSHG', '600965.XSHG')
找到匹配数据：('600975.XSHG', '002746.XSHE')
找到匹配数据：('002714.XSHE', '002234.XSHE')
找到匹配数据：('300106.XSHE', '002714.XSHE')
找到匹配数据：('002714.XSHE', '002714.XSHE')
找到匹配数据：('002234.XSHE', '300106.XSHE')
找到匹配数据：('300106.XSHE', '600965.XSHG')
找到匹配数据：('002299.XSHE', '002299.XSHE')
找到匹配数据：('002299.XSHE', '600975.XSHG')
找到匹配数据：('000735.XSHE', '000735.XSHE')
找到匹配数据：('300

行业内合计股票数为：8
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
找到匹配数据：('600759.XSHG', '600759.XSHG')
找到匹配数据：('600759.XSHG', '600256.XSHG')
找到匹配数据：('000968.XSHE', '601857.XSHG')
找到匹配数据：('600028.XSHG', '600028.XSHG')
找到匹配数据：('600759.XSHG', '000956.XSHE')
找到匹配数据：('000968.XSHE', '000968.XSHE')
找到匹配数据：('601857.XSHG', '000968.XSHE')
找到匹配数据：('000817.XSHE', '600028.XSHG')
找到匹配数据：('600759.XSHG', '000406.XSHE')
找到匹配数据：('600028.XSHG', '000968.XSHE')
找到匹配数据：('000956.XSHE', '600256.XSHG')
找到匹配数据：('600256.XSHG', '000817.XSHE')
找到匹配数据：('600759.XSHG', '601857.XSHG')
找到匹配数据：('601857.XSHG', '000406.XSHE')
找到匹配数据：('601857.XSHG', '000817.XSHE')
找到匹配数据：('000956.XSHE', '000968.XSHE')
找到匹配数据：('601857.XSHG', '600028.XSHG')
找到匹配数据：('600256.XSHG', '000956.XSHE')
找到匹配数据：('600256.XSHG', '600028.XSHG')
找到匹配数据：('000817.XSHE', '601857.XSHG')
找到匹配数据：('000956.XSHE', '600759.XSHG')
找到匹配数据：('601857.XSHG', '600759.XSHG')
找到匹配数据：('000406.XSHE', '600256.XSHG')
找到匹配数据：('600256.XSHG', '600759.XSHG')
找到匹配数据：('600256.XSHG', '6002

开始计算：B10
行业内合计股票数为：1
进展到第0
找到匹配数据：('603505.XSHG', '603505.XSHG')
开始计算：B11
行业内合计股票数为：16
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
进展到第11
进展到第12
进展到第13
进展到第14
进展到第15
找到匹配数据：('603619.XSHG', '600339.XSHG')
找到匹配数据：('002683.XSHE', '002683.XSHE')
找到匹配数据：('002629.XSHE', '002554.XSHE')
找到匹配数据：('300084.XSHE', '300084.XSHE')
找到匹配数据：('300157.XSHE', '300084.XSHE')
找到匹配数据：('603979.XSHG', '600871.XSHG')
找到匹配数据：('600583.XSHG', '300191.XSHE')
找到匹配数据：('600871.XSHG', '002554.XSHE')
找到匹配数据：('600339.XSHG', '600583.XSHG')
找到匹配数据：('002683.XSHE', '002554.XSHE')
找到匹配数据：('002683.XSHE', '300084.XSHE')
找到匹配数据：('300084.XSHE', '300157.XSHE')
找到匹配数据：('601808.XSHG', '002683.XSHE')
找到匹配数据：('603619.XSHG', '300157.XSHE')
找到匹配数据：('600871.XSHG', '002683.XSHE')
找到匹配数据：('002629.XSHE', '300191.XSHE')
找到匹配数据：('300191.XSHE', '601808.XSHG')
找到匹配数据：('002554.XSHE', '002683.XSHE')
找到匹配数据：('603619.XSHG', '603619.XSHG')
找到匹配数据：('300157.XSHE', '603619.XSHG')
找到匹配数据：('300084.XSHE', '300191.XSHE')
找到匹配数据：('3001

找到匹配数据：('000911.XSHE', '002702.XSHE')
找到匹配数据：('002157.XSHE', '000911.XSHE')
找到匹配数据：('002702.XSHE', '002124.XSHE')
找到匹配数据：('601952.XSHG', '600438.XSHG')
找到匹配数据：('002286.XSHE', '000639.XSHE')
找到匹配数据：('002557.XSHE', '000529.XSHE')
找到匹配数据：('000876.XSHE', '002557.XSHE')
找到匹配数据：('600438.XSHG', '600275.XSHG')
找到匹配数据：('000893.XSHE', '002695.XSHE')
找到匹配数据：('002852.XSHE', '300175.XSHE')
找到匹配数据：('600127.XSHG', '000895.XSHE')
找到匹配数据：('300175.XSHE', '600251.XSHG')
找到匹配数据：('000505.XSHE', '002124.XSHE')
找到匹配数据：('300138.XSHE', '002557.XSHE')
找到匹配数据：('000529.XSHE', '600737.XSHG')
找到匹配数据：('002515.XSHE', '600127.XSHG')
找到匹配数据：('600275.XSHG', '600275.XSHG')
找到匹配数据：('000895.XSHE', '000911.XSHE')
找到匹配数据：('000048.XSHE', '300175.XSHE')
找到匹配数据：('000529.XSHE', '002515.XSHE')
找到匹配数据：('002220.XSHE', '000911.XSHE')
找到匹配数据：('603517.XSHG', '002548.XSHE')
找到匹配数据：('002702.XSHE', '600438.XSHG')
找到匹配数据：('002567.XSHE', '002702.XSHE')
开始计算：C14
行业内合计股票数为：43
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10

...

找到匹配数据：('002087.XSHE', '000955.XSHE')
找到匹配数据：('002516.XSHE', '300577.XSHE')
找到匹配数据：('002144.XSHE', '600232.XSHG')
找到匹配数据：('002015.XSHE', '000158.XSHE')
找到匹配数据：('000726.XSHE', '000611.XSHE')
找到匹配数据：('000158.XSHE', '300577.XSHE')
找到匹配数据：('000955.XSHE', '600448.XSHG')
找到匹配数据：('600448.XSHG', '601339.XSHG')
找到匹配数据：('000850.XSHE', '002516.XSHE')
找到匹配数据：('600689.XSHG', '600070.XSHG')
找到匹配数据：('002015.XSHE', '300577.XSHE')
找到匹配数据：('000158.XSHE', '002144.XSHE')
找到匹配数据：('600689.XSHG', '002144.XSHE')
找到匹配数据：('002293.XSHE', '600987.XSHG')
找到匹配数据：('600156.XSHG', '002070.XSHE')
找到匹配数据：('600630.XSHG', '600493.XSHG')
找到匹配数据：('002042.XSHE', '002042.XSHE')
找到匹配数据：('002083.XSHE', '603889.XSHG')
找到匹配数据：('601339.XSHG', '000158.XSHE')
找到匹配数据：('002087.XSHE', '002015.XSHE')
找到匹配数据：('601339.XSHG', '600232.XSHG')
找到匹配数据：('603889.XSHG', '002144.XSHE')
找到匹配数据：('600070.XSHG', '000955.XSHE')
找到匹配数据：('600987.XSHG', '601339.XSHG')
找到匹配数据：('000158.XSHE', '002042.XSHE')
找到匹配数据：('002397.XSHE', '600689.XSHG')
找到匹配数据：('600

行业内合计股票数为：11
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
找到匹配数据：('002674.XSHE', '603001.XSHG')
找到匹配数据：('603001.XSHG', '600439.XSHG')
找到匹配数据：('600439.XSHG', '603608.XSHG')
找到匹配数据：('603116.XSHG', '603608.XSHG')
找到匹配数据：('600439.XSHG', '603116.XSHG')
找到匹配数据：('002674.XSHE', '002674.XSHE')
找到匹配数据：('603001.XSHG', '603001.XSHG')
找到匹配数据：('600439.XSHG', '002674.XSHE')
找到匹配数据：('002494.XSHE', '603001.XSHG')
找到匹配数据：('603477.XSHG', '002494.XSHE')
找到匹配数据：('603608.XSHG', '603116.XSHG')
找到匹配数据：('603477.XSHG', '603116.XSHG')
找到匹配数据：('603001.XSHG', '603608.XSHG')
找到匹配数据：('603608.XSHG', '603477.XSHG')
找到匹配数据：('603608.XSHG', '002674.XSHE')
找到匹配数据：('603608.XSHG', '603001.XSHG')
找到匹配数据：('603001.XSHG', '002494.XSHE')
找到匹配数据：('002494.XSHE', '603557.XSHG')
找到匹配数据：('600439.XSHG', '603001.XSHG')
找到匹配数据：('603557.XSHG', '002494.XSHE')
找到匹配数据：('600439.XSHG', '002494.XSHE')
找到匹配数据：('300591.XSHE', '002674.XSHE')
找到匹配数据：('603608.XSHG', '600439.XSHG')
找到匹配数据：('002494.XSHE', '600439.XSHG')
找到匹配数据：(

行业内合计股票数为：29
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
进展到第11
进展到第12
进展到第13
进展到第14
进展到第15
进展到第16
进展到第17
进展到第18
进展到第19
进展到第20
进展到第21
进展到第22
进展到第23
进展到第24
进展到第25
进展到第26
进展到第27
进展到第28
找到匹配数据：('600963.XSHG', '603165.XSHG')
找到匹配数据：('002012.XSHE', '002012.XSHE')
找到匹配数据：('002067.XSHE', '000699.XSHE')
找到匹配数据：('600069.XSHG', '000815.XSHE')
找到匹配数据：('002228.XSHE', '002303.XSHE')
找到匹配数据：('002565.XSHE', '600235.XSHG')
找到匹配数据：('000488.XSHE', '600963.XSHG')
找到匹配数据：('002799.XSHE', '600433.XSHG')
找到匹配数据：('600963.XSHG', '000488.XSHE')
找到匹配数据：('000815.XSHE', '000576.XSHE')
找到匹配数据：('000815.XSHE', '603022.XSHG')
找到匹配数据：('000488.XSHE', '600103.XSHG')
找到匹配数据：('600308.XSHG', '002228.XSHE')
找到匹配数据：('603165.XSHG', '600567.XSHG')
找到匹配数据：('000833.XSHE', '600966.XSHG')
找到匹配数据：('000699.XSHE', '600356.XSHG')
找到匹配数据：('000699.XSHE', '000833.XSHE')
找到匹配数据：('002511.XSHE', '002303.XSHE')
找到匹配数据：('600103.XSHG', '600433.XSHG')
找到匹配数据：('002831.XSHE', '000815.XSHE')
找到匹配数据：('600567.XSHG', '002565.XSH

开始计算：C23
行业内合计股票数为：12
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
进展到第5
进展到第6
进展到第7
进展到第8
进展到第9
进展到第10
进展到第11
找到匹配数据：('002191.XSHE', '002803.XSHE')
找到匹配数据：('000812.XSHE', '002117.XSHE')
找到匹配数据：('002117.XSHE', '002117.XSHE')
找到匹配数据：('002599.XSHE', '002836.XSHE')
找到匹配数据：('002803.XSHE', '002191.XSHE')
找到匹配数据：('603058.XSHG', '002836.XSHE')
找到匹配数据：('603429.XSHG', '002229.XSHE')
找到匹配数据：('601515.XSHG', '002803.XSHE')
找到匹配数据：('603429.XSHG', '002803.XSHE')
找到匹配数据：('002117.XSHE', '000812.XSHE')
找到匹配数据：('002117.XSHE', '002836.XSHE')
找到匹配数据：('601515.XSHG', '000812.XSHE')
找到匹配数据：('002117.XSHE', '002803.XSHE')
找到匹配数据：('603429.XSHG', '600836.XSHG')
找到匹配数据：('002191.XSHE', '002117.XSHE')
找到匹配数据：('002803.XSHE', '002803.XSHE')
找到匹配数据：('002117.XSHE', '002191.XSHE')
找到匹配数据：('000812.XSHE', '002229.XSHE')
找到匹配数据：('601515.XSHG', '002836.XSHE')
找到匹配数据：('002836.XSHE', '002117.XSHE')
找到匹配数据：('000812.XSHE', '002836.XSHE')
找到匹配数据：('002803.XSHE', '002229.XSHE')
找到匹配数据：('002599.XSHE', '601515.XSHG')
找到匹配数据：('002229.XSHE', '002229

行业内合计股票数为：237
进展到第0
进展到第1
进展到第2
进展到第3
进展到第4
